In [ ]:
import gensim
from load_data import load_media
from load_data import load_users
from load_data import train_test_pipeline
import random

random.seed(4)

Load data sets.

In [ ]:
graph = load_users('../users.csv')
media, hashtags = load_media('../media.csv')

Generate embeddings using word2vec algorithm. Only do this if you have not generated a model before.

In [ ]:
# generate_embeddings(('../media.csv', './tag2vec64dIterator.model', threads=1)
# split_nway('../mini_media.csv', './mini_s10.dat')
# generate_embeddings('../mini_media.csv', './tag2vec_mini.model', dist_path='../mini_s10.txt', cur_set=0)
# eval_predictions('../mini_media.csv', '../mini_s10.txt', './tag2vec_mini.model', 0)
train_test_pipeline('../mini_media.csv', './mini_dist.dat', './64d', 10)

Load embeddings in memory.

In [ ]:
model = gensim.models.Word2Vec.load('./tag2vec64dFixedSkipgram.model')

In [ ]:
model.wv.most_similar(positive=['flashpackerborneo'], topn=10)

In [ ]:
model.wv.similarity(w1="california",w2="sanfrancisco")